# Complete reduction script - for spectra

## Configuration

In [8]:
#dir = 'e:/Astro/Captures/20231008_Void/'
dir = 'E:/Astro/Captures/20181020_spectro/'
#bias = 'Bias*.fit'
bias = 'offset-*.fit'
#darks = 'dark*.fit'
darks = 'dark*.fit'
flats = 'flat*.fit'
#flats = 'flat*.fit'
sciences = 'bwvull-*.fit'
#sciences = 'deneb*.fit'

masterbias = 'masterbias.fit'
masterdark = 'masterdark.fit'
masterflat = 'masterflat.fit'
sciencedata = 'bwvull-star.fit'

overwrite_masters = True         # recreate masters or not ?
memory_limit = 1e9               # how much memory (bytes) to allocate to ccdproc combine ?

camera_electronic_gain = 1.2     # asi 183mm
camera_readout_noise = 2.2       # asi 183mm

## Imports libs

In [9]:
%matplotlib widget
import warnings, fnmatch, os
from time import gmtime, strftime
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table, QTable
from astropy import units as u
from astropy.nddata import CCDData, StdDevUncertainty
from astropy.stats import mad_std
from astropy.io import fits
from astropy.utils.exceptions import AstropyWarning

from ccdproc import Combiner, combine, subtract_bias, subtract_dark, flat_correct
from ccdproc import trim_image, Combiner, ccd_process, cosmicray_median

from align_combine import align_and_combine

warnings.simplefilter('ignore', category=AstropyWarning)
#warnings.simplefilter('ignore', category=FITSFixedWarning)


## Create logger

In [10]:
from logger_utils import logger, handler
handler.show_logs()
handler.clear_logs()
logger.setLevel('INFO')


Output(layout=Layout(border_bottom='1px solid grey', border_left='1px solid grey', border_right='1px solid gre…

## Create masters

In [18]:
### prepare filenames lists
bias_files = [dir + f for f in fnmatch.filter(os.listdir(dir), bias)]
dark_files = [dir + f for f in fnmatch.filter(os.listdir(dir), darks)]
flat_files = [dir + f for f in fnmatch.filter(os.listdir(dir), flats)]
science_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), sciences)]
logger.info('bias files : ' + repr(bias_files))
logger.info('dark files : ' + repr(dark_files))
logger.info('flat files : ' + repr(flat_files))
logger.info('science files : ' + repr(science_files))

### create masterbias
bias_list =  []
bias_master = np.array([])
if (os.path.exists(dir + masterbias)) and (overwrite_masters is False):
    logger.info(masterbias + ' not (re)created')
else:
    logger.info('masterbias combine...')
    bias_list = [CCDData.read(f, unit = u.adu ) for f in bias_files]         
    #_bias_master = combine(bias_list, method = 'median', dtype = np.float32, mem_limit = memory_limit)   
    _bias_master = combine(bias_list,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std, # dtype = np.float32,
                             mem_limit = memory_limit
                            )

    bias_master = CCDData(_bias_master.data.astype('float32'), unit = u.adu , header = _bias_master.header)
    bias_master.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
    bias_master.meta['combined'] = True
    bias_master.meta['history'] = 'combined from {} bias file(s)'.format(len(bias_files))
    bias_master.write(dir + masterbias, overwrite=True)
    logger.info('masterbias created - combined from {} bias file(s)'.format(len(bias_files)))
    bias_list = None
    _bias_master = None    

### create masterdark
darks_list = []
dark_master = np.array([])
if (os.path.exists(dir + masterdark)) and (overwrite_masters is False):
    logger.info(masterdark + ' not (re)created')
else:
    for dark_file in dark_files:
        logger.info('masterbias sub {} ...'.format(dark_file))
        _dark_data = CCDData.read(dark_file, unit = u.adu )
        #_dark_sub = subtract_bias(_dark_data, bias_master)
        _dark_sub = ccd_process(_dark_data, 
            oscan = None, 
            gain_corrected = False, 
            trim = None, 
            error = False,
            gain = camera_electronic_gain*u.electron/u.adu ,
            readnoise = camera_readout_noise*u.electron,
            master_bias = bias_master,
            dark_frame = None,
            master_flat = None,
            exposure_key = 'EXPTIME',
            exposure_unit = u.second,
            dark_scale = False)
      
        dark_sub = CCDData(_dark_sub.data.astype('float32'), unit = u.adu , header = _dark_data.header)
        darks_list.append(dark_sub)
        _dark_data = None
        _dark_sub = None
    
    logger.info('masterdark combine...')
    #dark_master = combine(darks_list, method='median', dtype=np.uint32, mem_limit = memory_limit)
    _dark_master = combine(darks_list,
                                 method='average',
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std, dtype=np.float32,
                                 mem_limit=memory_limit
                                )
    
    dark_master = CCDData(_dark_master.data.astype('float32'), unit = u.adu , header = _dark_master.header)
    dark_master.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
    dark_master.meta['combined'] = True
    dark_master.meta['history'] = 'masterbias sub + combined from {} dark file(s)'.format(len(dark_files))
    dark_master.write(dir + masterdark, overwrite=True)
    logger.info('masterdark created - bias substracted + combined from {} dark file(s)'.format(len(dark_files)))
    darks_list = None
    _dark_master = None

### create masterflat
flats_list = []
flat_master = np.array([])

def inv_median(a):
    return 1 / np.median(a)
    
if (os.path.exists(dir + masterflat)) and (overwrite_masters is False):
    logger.info(masterflat + ' not (re)created')
else:
    for flat_file in flat_files:
        logger.info('masterbias sub {} ...'.format(flat_file))
        _flat_data = CCDData.read(flat_file, unit = u.adu )
        _flat_sub = subtract_bias(_flat_data, bias_master)
        flat_sub = CCDData(_flat_sub.data.astype('float32'), unit = u.adu , header = _flat_data.header)
        flats_list.append(flat_sub)
        _flat_data = None
        _flat_sub = None
    
    logger.info('masterflat combine...')
    #flat_master = combine(flats_list, method='median', dtype=np.float32, mem_limit = memory_limit)
    _flat_master = combine(flats_list,
                                 method='average', #scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std, dtype=np.float32,
                                 mem_limit=memory_limit
                                )
    flat_master = CCDData(_flat_master.data.astype('float32'), unit = u.adu , header = _flat_master.header)
    flat_master.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
    flat_master.meta['combined'] = True
    flat_master.meta['history'] = 'bias substracted + combined from {} flat file(s)'.format(len(flat_files))
    flat_master.write(dir + masterflat, overwrite=True)
    logger.info('masterflat created : bias substracted + combined from {} flat file(s)'.format(len(flat_files)))
    flats_list = None
    _flat_master = None



INFO: splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]


## Reduce science data

In [15]:
if overwrite_masters is False:
    bias_master = CCDData.read(dir + masterbias, unit = u.adu )
    dark_master = CCDData.read(dir + masterdark, unit = u.adu )
    flat_master = CCDData.read(dir + masterflat, unit = u.adu )

"""
### process science files - individual science file process version
for sci_file in science_files:
    logger.info('masterbias sub {} ...'.format(sci_file))
    #sci_data = CCDData.read(sci_file, unit = u.adu )
    sci_sub_bias = subtract_bias(CCDData.read(sci_file, unit = u.adu ), bias_master)
    #sci_data = None
    logger.info('masterdark sub {} ...'.format(sci_file))
    sci_sub_dark = subtract_dark(sci_sub_bias, dark_master, scale = True, exposure_time='EXPTIME', exposure_unit=u.second)
    sci_sub_bias = None
    logger.info('masterflat divide {} ...'.format(sci_file))
    sci_div_flat = flat_correct(sci_sub_dark, flat_master)
    sci_sub_dark = None
    sci_ok = CCDData(sci_div_flat.data.astype('float32'), unit = u.adu , header = sci_div_flat.header)
    sci_div_flat = None
    logger.info('writing science file (-ppr) {}'.format(sci_file))
    sci_ok.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
    sci_ok.meta['history'] = 'bias substracted + dark substrated + flat divide'
    sci_ok.write(Path(sci_file).with_stem(f"{'ppr'}-{Path(sci_file).stem}"), overwrite=True)
"""

### process science files - ccd_process version
for sci_file in science_files:
    logger.info('processing (masterbias sub + masterdark sub + masterflat divide) {} ...'.format(sci_file))
    sci_data = CCDData.read(sci_file, unit = u.adu )
    sci_data_processed = ccd_process(sci_data, 
            oscan = None, 
            gain_corrected = False, 
            trim = None, 
            error = False,
            gain = camera_electronic_gain*u.electron/u.adu ,
            readnoise = camera_readout_noise*u.electron,
            master_bias = bias_master,
            dark_frame = dark_master,
            master_flat = flat_master,
            exposure_key = 'EXPTIME',
            exposure_unit = u.second,
            dark_scale = True)
    sci_ok = CCDData(sci_data_processed.data.astype('float32'), unit = u.adu , header = sci_data.header)
    sci_data = None
    logger.info('writing science datafile (-ppr) {}'.format(sci_file))
    sci_ok.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
    sci_ok.meta['history'] = 'bias substracted + dark substrated + flat divide'
    sci_ok.write(Path(sci_file).with_stem(f"{'ppr'}-{Path(sci_file).stem}"), overwrite=True)

### align images and combine using a SUM
science_files_ppr  = [dir + f for f in fnmatch.filter(os.listdir(dir), 'ppr-*' + sciences)]
logger.info('science files aligning and combining ... : ' + repr(science_files_ppr))
sci_aligned = align_and_combine(science_files_ppr, np.sum)
sci_master = CCDData(sci_aligned, unit = u.adu , header = sci_ok.header)
sci_master.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
sci_master.meta['combined'] = True
sci_master.meta['history'] = 'aligned + combined from {} sciences file(s)'.format(len(science_files))
sci_master.write(dir + 'ppr-' + sciencedata, overwrite=True)
logger.info('aligned + combined from {} sciences file(s)'.format(len(science_files)))

"""
### remove cosmic
logger.info('removing cosmic rays...')
#sci_cleaned = cosmicray_median(sci_master, thresh = 5, mbox=11, rbox=11, gbox=5, error_image = np.ones(sci_master.shape))
sci_cleaned = cosmicray_median(sci_master, thresh = 3, mbox=5, rbox=5, gbox=5, error_image = np.ones(sci_master.shape))
sci_cleaned.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
sci_cleaned.meta['history'] = 'cosmicray median removed'
sci_cleaned.write(dir + 'final-' + sciencedata, overwrite=True)
logger.info('final-' + sciencedata + ' created')
"""

'\n### remove cosmic\nlogger.info(\'removing cosmic rays...\')\n#sci_cleaned = cosmicray_median(sci_master, thresh = 5, mbox=11, rbox=11, gbox=5, error_image = np.ones(sci_master.shape))\nsci_cleaned = cosmicray_median(sci_master, thresh = 3, mbox=5, rbox=5, gbox=5, error_image = np.ones(sci_master.shape))\nsci_cleaned.meta[\'DATE\'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())\nsci_cleaned.meta[\'history\'] = \'cosmicray median removed\'\nsci_cleaned.write(dir + \'final-\' + sciencedata, overwrite=True)\nlogger.info(\'final-\' + sciencedata + \' created\')\n'

In [ ]:
logger.info('removing cosmic rays...')
sci_cleaned = cosmicray_median(sci_aligned, thresh = 5, mbox=11, rbox=11, gbox=11, error_image = None)
sci_cleaned.meta['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
sci_cleaned.meta['history'] = 'cosmicray median removed'
sci_cleaned.write(dir + 'final-' + sciencedata, overwrite=True)
logger.info('final-' + sciencedata + ' created')


In [ ]:
dark_300 = dark_master.multiply(1.2 * u.electron / u.adu).divide(300 * u.second)
plt.figure(figsize=(8, 6))

plt.hist(dark_300.data.flatten(), bins=500, density=False, label='300 sec dark', alpha=0.4)
plt.xlabel('dark current, $e^-$/sec')
plt.ylabel('Number of pixels')
plt.loglog()
plt.legend();

In [ ]:
hot_pixels = (dark_300.data > 10)
plt.figure(figsize=(10, 10))
plt.plot(dark_300.data[hot_pixels].flatten(), '.', alpha=0.2, label='Data')
plt.xlabel("dark current ($e^-$/sec), 90 sec exposure time")
plt.ylabel("dark current ($e^-$/sec), 1000 sec exposure time")
plt.plot([0, 100], [0, 100], label='Ideal relationship')
plt.grid()
plt.xlim(0.5, 10)
plt.ylim(0.5, 10)
plt.legend();

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(sci_master.data, origin='lower', aspect='auto', cmap=plt.cm.magma)
plt.title('sci_master')
plt.clim(np.percentile(sci_master.data, (5, 98)))
cb = plt.colorbar()

plt.figure(figsize=(10,6))
plt.imshow(sci_cleaned[0], origin='lower', aspect='auto', cmap=plt.cm.magma)
plt.title('sci_cleaned')
plt.clim(np.percentile(sci_cleaned[0], (5, 98)))
cb = plt.colorbar()

In [ ]:
master_bias = CCDData.read(dir + masterbias, unit = u.adu)
master_dark = CCDData.read(dir + masterdark, unit = u.adu)
master_flat = CCDData.read(dir + masterflat, unit = u.adu)
science_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), sciences)]
ppr_science_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), 'ppr-' + sciences)]
one_science = CCDData.read(science_files[0], unit = u.adu)
one_science_ppr = CCDData.read(ppr_science_files[0], unit = u.adu)
master_science = CCDData.read(dir + 'ppr-' +sciencedata, unit = u.adu)
#master_science_cosmic = CCDData.read(dir + 'final-' + sciencedata, unit = u.adu)

#cmap = plt.cm.magma
colormap = plt.cm.magma
cuts = (5, 99.)

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, _)) = plt.subplots(4, 2,figsize=(15, 15))

im1 = ax1.imshow(master_bias, origin='lower', aspect='auto', cmap=colormap)
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)
im1.set_clim(np.percentile(master_bias, cuts))
ax1.set_title('master_bias')
cb = plt.colorbar(im1)

im2 = ax2.imshow(master_dark, origin='lower', aspect='auto', cmap=colormap)
ax2.xaxis.set_visible(False)
ax2.yaxis.set_visible(False)
im2.set_clim(np.percentile(master_dark, cuts))
ax2.set_title('master_dark')
cb = plt.colorbar(im2)

im3 = ax3.imshow(master_flat, origin='lower', aspect='auto', cmap=colormap)
ax3.xaxis.set_visible(False)
ax3.yaxis.set_visible(False)
im3.set_clim(np.percentile(master_flat, cuts))
ax3.set_title('master_flat')
cb = plt.colorbar(im3)

im4 = ax4.imshow(one_science, origin='lower', aspect='auto', cmap=colormap)
ax4.xaxis.set_visible(False)
ax4.yaxis.set_visible(False)
im4.set_clim(np.percentile(one_science, cuts))
ax4.set_title('one_science_raw')
cb = plt.colorbar(im4)

im5 = ax5.imshow(one_science_ppr, origin='lower', aspect='auto', cmap=colormap)
ax5.xaxis.set_visible(False)
ax5.yaxis.set_visible(False)
im5.set_clim(np.percentile(one_science_ppr, cuts))
ax5.set_title('one_science_processed')
cb = plt.colorbar(im5)

im6 = ax6.imshow(master_science, origin='lower', aspect='auto', cmap=colormap)
ax6.xaxis.set_visible(False)
ax6.yaxis.set_visible(False)
im6.set_clim(np.percentile(master_science, cuts))
ax6.set_title('master_science')
cb = plt.colorbar(im6)
"""
im7 = ax7.imshow(master_science_cosmic[0], origin='lower', aspect='auto', cmap=colormap)
ax7.xaxis.set_visible(False)
ax7.yaxis.set_visible(False)
im7.set_clim(np.percentile(master_science_cosmic, cuts))
ax7.set_title('master_science_cosmicray_removed')
cb = plt.colorbar(im7)
"""
_.cla()
_.clear()



In [ ]:
from align_combine import align_and_combine
dir = 'e:/Astro/Captures/20231007_Void/'
#dir = 'E:/Astro/Captures/20181020_spectro/'

#sci_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), 'bwvull-*.fit')]
sci_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), '10lac-300s-*.fit')]

sci_datas = [CCDData.read(f, unit = u.adu) for f in sci_files]
sci_data = np.sum(sci_datas, axis = 0, dtype = np.float32)


plt.figure(figsize=(10,6))
plt.imshow(sci_data, origin='lower', aspect='auto', cmap=plt.cm.magma)
plt.title('sci_data')
plt.clim(np.percentile(sci_data, (5, 99.95)))
cb = plt.colorbar()


sci_aligned = align_and_combine(sci_files, np.sum)
plt.figure(figsize=(10,6))
plt.imshow(sci_aligned, origin='lower', aspect='auto', cmap=plt.cm.magma)
plt.title('sci_aligned')
plt.clim(np.percentile(sci_aligned, (5, 99.95)))
cb = plt.colorbar()



In [ ]:
#sci_data.header['EXPOSURE'] = sci_data.header['EXPTIME']  # for dark subtraction

sci_data = ccdproc.ccd_process(sci_data, oscan=None, gain_corrected=False,
                           trim=None,
                           gain=2.0*u.electron/u.adu,
                           readnoise=5*u.electron,
                           dark_frame=master_dark,
                           master_bias = master_bias,
                           exposure_key='EXPTIME',
                           exposure_unit=u.second,
                           dark_scale=True,
                           master_flat=master_flat)


In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(sci_data, origin='lower', aspect='auto', cmap=plt.cm.magma)
plt.title('sci_data')
plt.clim(np.percentile(sci_data, (5, 99.99)))
cb = plt.colorbar()


In [ ]:
#newccd = ccdproc.cosmicray_median(nccd, thresh=5, mbox=11, rbox=11, gbox=5) 
#cr_cleaned = ccdproc.cosmicray_lacosmic(nccd, sigclip=1)
cr_cleaned = ccdproc.cosmicray_median(sci_data, thresh = 3, mbox=5, rbox=5, gbox=5, error_image = None) #np.ones(sci_data.shape))
cr_cleaned

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(cr_cleaned[0], origin='lower', aspect='auto', cmap=plt.cm.magma)
plt.title('cr_cleaned')
plt.clim(np.percentile(cr_cleaned, (5, 99.99)))
cb = plt.colorbar()

In [ ]:
from specreduce import tracing, background, extract
sci_tr = tracing.FitTrace(nccd,  peak_method='max')   #FitTrace(image, peak_method='gaussian', guess=trace_pos)
bg = background.Background.two_sided(nccd, sci_tr, separation=50, width=50) 
extract = extract.BoxcarExtract(nccd - bg, sci_tr, width = 5)
sci_spectrum = extract()


In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(sci_data, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.imshow(bg.bkg_wimage, origin='lower', aspect='auto', cmap=plt.cm.gray, alpha=0.1)
plt.plot(sci_tr.trace , color='r')
plt.plot(sci_tr.trace+extract.width , color='g', linestyle='dashed', alpha=0.5)
plt.plot(sci_tr.trace-extract.width , color='g', linestyle='dashed', alpha=0.5)

cb = plt.colorbar()



In [ ]:
plt.figure(figsize=(10,6))
plt.plot(sci_spectrum.flux, color='r',  linewidth = '0.6')


In [ ]:
dir = 'e:/Astro/Captures/20231008_Void/'
neon_data = CCDData.read(dir + 'neon-15s-2.fit', unit = u.adu)
plt.figure(figsize=(10,6))
plt.imshow(neon_data, origin='lower', aspect='auto', cmap=plt.cm.grey)
plt.title('neon_data')
plt.clim(np.percentile(sci_data, (10, 90)))
cb = plt.colorbar()

In [ ]:
from specreduce import tracing, background, extract
#specreduce.tracing.ArrayTrace(image: NDData, trace: ndarray)
neon_tr = tracing.ArrayTrace(neon_data, [1900, 2000])   #FitTrace(image, peak_method='gaussian', guess=trace_pos)
#bg = background.Background.two_sided(nccd, sci_tr, separation=50, width=50) 
extract = extract.BoxcarExtract(neon_data , neon_tr, width = 5)
neon_spectrum = extract()
plt.figure(figsize=(10,6))
plt.plot(neon_spectrum.flux, color='b',  linewidth = '0.6')



In [ ]:
import astropy.units as u
from specreduce import WavelengthCalibration1D


pixels = [868, 1276, 2342, 3635, 4263]*u.pix
wavelength = [6506.53, 6532.88, 6598.95, 6678.28, 6717.04]*u.AA
line_list = QTable([pixels, wavelength], names=["pixel_center", "wavelength"])
test_cal = WavelengthCalibration1D(neon_spectrum, matched_line_list=line_list)
print(test_cal.residuals )
print(test_cal.fitted_model )
neon_calibrated_spectrum = test_cal.apply_to_spectrum(neon_spectrum)
sci_calibrated_spectrum = test_cal.apply_to_spectrum(sci_spectrum)

plt.figure(figsize=(10,6))
plt.plot(neon_calibrated_spectrum.spectral_axis, neon_calibrated_spectrum.flux)  

plt.figure(figsize=(10,6))
plt.plot(sci_calibrated_spectrum.spectral_axis, sci_calibrated_spectrum.flux)  
